In [ ]:
!pip install mediapipe 

In [ ]:
!wget https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/latest/face_landmarker.task

In [ ]:
model_path = '/kaggle/working/face_landmarker.task'

In [ ]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.protobuf import timestamp_pb2
from statistics import mean

In [ ]:
video_path = '/kaggle/input/trialvideo/sample.mp4'
image_path = '/kaggle/input/trialvideo/ring-xs.png'
output_path = 'output_video.mp4'

In [ ]:
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [ ]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

In [ ]:
# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('face landmarker result: {}'.format(result))

In [ ]:
def build_overlay(frame,img,x,y):
    ''' Creates the overlay for the given frame and return the frame for overlay which will be
        weighted added to the original frame
    '''
    h,w,_ = frame.shape
    overlay = np.zeros((h,w,3),dtype='uint8')
    
    img_h,img_w,_ = img.shape
    try:
        for i in range(0,img_h):
            for j in range(0,img_w):
                if(img[i,j][0]!=0 or img[i,j][1]!=0 or img[i,j][2]!=0 ):
                    overlay[x+i,y+j] = img[i,j]
    except IndexError:
        pass
    return overlay 

In [ ]:
mode='img'

In [ ]:
landmarker = FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(video_path)
ring_image = cv2.imread(image_path)
ring_image = cv2.resize(ring_image, (32,32))

# Configure output video properties
codec = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = 0  # Set to 0 to use the same width as the input video
frame_height = 0  # Set to 0 to use the same height as the input video
fps = 30  # Set the desired output frame rate

# Check if video was successfully opened
if not cap.isOpened():
    print("Failed to open the video file.")
    exit()

# Get input video properties
if frame_width == 0:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
if frame_height == 0:
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer object
out = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))

# Configure output video properties
codec = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = 0  # Set to 0 to use the same width as the input video
frame_height = 0  # Set to 0 to use the same height as the input video
fps = 30  # Set the desired output frame rate

# Get input video properties
if frame_width == 0:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
if frame_height == 0:
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer object
out = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))
n_frames = 0
n_frames_wo_landmarks = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    n_frames += 1
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=im)
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
    results = landmarker.detect_for_video(mp_image,mp.Timestamp.from_seconds(timestamp).microseconds())
    if len(results.face_landmarks) == 0:
        annotated_image = mp_image.numpy_view()
        annotated_image =  cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        out.write(annotated_image)
        n_frames_wo_landmarks += 1
        continue
#     annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), results)
#     base_vector1 = np.array([1, 0, 0])
#     base_vector2 = np.array([0, 1, 0])
#     image_plane = np.cross(base_vector2, -base_vector1)
    image_plane = np.array([0, 0, 1])
    result1 = results.face_landmarks[0][440]
    result2 = results.face_landmarks[0][363]
    result3 = results.face_landmarks[0][360]
    vec1 = [result1.x - result2.x, result1.y - result2.y, result1.z - result2.z]
    vec2 = [result2.x - result3.x, result2.y - result3.y, result2.z - result3.z]
    triangle_plane = np.cross(vec1, vec2)
    
    occlusion_val = np.dot(image_plane, triangle_plane)
    
    if occlusion_val > 0:
        coords = (int(frame_width*mean([result1.x,result2.x,result3.x])), int(frame_height*mean([result1.y,result2.y,result3.y])))
        if mode=='img':
            overlay = build_overlay(mp_image.numpy_view(),ring_image,coords[1],coords[0])
            annotated_image = cv2.addWeighted(overlay,1,mp_image.numpy_view(),1,0.0)
        else: 
            annotated_image = cv2.circle(mp_image.numpy_view(), coords, radius=2, color=(255,0,0), thickness=2)
    else:
        annotated_image = mp_image.numpy_view()
    annotated_image =  cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    out.write(annotated_image)
cap.release()
out.release()

print("Total Frames:", n_frames)
print("Frames Without Landmarks,", n_frames_wo_landmarks)
print("Bad Frames", n_frames_wo_landmarks/n_frames*100)